## Import libraries

In [ ]:
import sys
sys.path.append('../Src/')
from data import *
from transfer_learning import *
from test_functions import *
from layers import *
from utils import *
from loss import *
from metric import *
from results import *
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers

In [ ]:
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

# Reference model

In [ ]:
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = \
    read_df_with_transfer_learning_subset_fewerDomainFeatures( \
        metadata_names=['age','Temperature','Precipitation3Days'], \
        otu_filename='../Datasets/otu_table_all_80.csv', \
        metadata_filename='../Datasets/metadata_table_all_80.csv')

In [ ]:
data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

In [ ]:
data_microbioma_train.shape

In [ ]:
# Train the selected model (the best one from those with the smallest latent space (10)): no.351
experiment_metrics, models, results = perform_experiment_2(cv_folds=0, 
                        epochs=100, 
                        batch_size=64, 
                        learning_rate=0.001, 
                        optimizer=optimizers.Adam,
                        learning_rate_scheduler=None,
                        input_transform=Percentage,
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=10, 
                        layers=[512,256],
                        activation='tanh', 
                        activation_latent='tanh', 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=None,
                        show_results=True, 
                        device='/CPU:0')

### To get encoders and decoders to use in domain->latent model

In [ ]:
model, encoder, _ ,decoder = models[0]

In [ ]:
df_domain_train.shape

### To predict latent space for samples in domain->latent model

In [ ]:
latent_train = encoder.predict(data_microbioma_train)
latent_test = encoder.predict(data_microbioma_test)

### To build model to predict latent space 

In [ ]:
def model_fn_latent():
    in_layer = layers.Input(shape=(3,))
    net = layers.Dense(128, activation='tanh')(in_layer)
    net = layers.Dense(64, activation='tanh')(net)
    net = layers.Dense(32, activation='tanh')(net)
    net = layers.Dense(16, activation='tanh')(net)
    out_layer = layers.Dense(latent_train.shape[1], activation=None)(net) # 'tanh already'
    model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
    model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(),
                 metrics=[tf.keras.metrics.MeanSquaredError()])
    return model

In [ ]:
result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                            latent_train,
                            latent_train,
                            data_domain_train,
                            data_domain_train,
                            epochs=100,
                            batch_size=16,
                            verbose=-1)

In [ ]:
predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

In [ ]:
#save_predictions(predictions, 'experiment_testSet_AE_OTUlatent_3var.txt')

### Save models

In [ ]:
encoder.save('../Results/Models/encoder_biome.h5')
decoder.save('../Results/Models/decoder.h5')
model_latent.save('../Results/Models/encoder_domain_model_latent.h5')

In [ ]:
encoder_biome = encoder
encoder_domain = model_latent

# Predictions

### Save predictions in files for external use (R plots)

In [ ]:
df_microbioma_test.T.to_csv('../Results/otus_original_test.tsv', index=True, header=True, sep='\t')

In [ ]:
def save_predicted_otu_table_and_latent(pred,pred_latent,sample_names,otu_names,suffix=''):
    df_otu = pd.DataFrame(pred, index=sample_names, columns=otu_names)
    df_otu.T.to_csv('../Results/otus_'+suffix+'.tsv', index=True, header=True, sep='\t')

    df_latent = pd.DataFrame(pred_latent, index=sample_names)
    df_latent.T.to_csv('../Results/latent_'+suffix+'.tsv', index=True, sep='\t')
    
    return df_otu, df_latent

#### Run prediction test set from microbiome, i.e. reconstructed

In [ ]:
# Input only domain (i.e. environmental features)
pred_latent_biome = encoder_biome.predict(data_microbioma_test)
pred_biome = decoder.predict(pred_latent_biome)

In [ ]:
_, _ = save_predicted_otu_table_and_latent(pred_biome,pred_latent_biome,df_microbioma_test.index,df_microbioma_test.columns,'reconstAEfromBiome')

#### Run prediction test set from domain (i.e. environmental features)

In [ ]:
# Input only domain (i.e. environmental features)
pred_latent = encoder_domain.predict(data_domain_test)
pred_domain = decoder.predict(pred_latent)

In [ ]:
df_pred_otu, df_pred_latent = save_predicted_otu_table_and_latent(pred_domain,pred_latent,df_microbioma_test.index,df_microbioma_test.columns,'predFromDomain')

# Compute metrics by OTU

#### Compute Relative Squared Error per OTU

In [ ]:
# Absolute abundance transformed to TSS (with epsilon=1E-6)
def transform_to_rel_abundance(dataset):
    epsilon=1E-6
    sum_per_sample = dataset.sum(axis=1)
    num_samples = sum_per_sample.shape
    num_OTUs = np.shape(dataset)[-1] 
    sum_per_sample = sum_per_sample + (num_OTUs * epsilon)
    dividend=dataset+epsilon
    dataset_rel_abund = np.divide(dividend,sum_per_sample[:,None])
    #display(Markdown("{}</p>".format(np.array2string(actual_array,precision=6,floatmode='fixed'))))
    #actual_array.sum(axis=1)
    return dataset_rel_abund

In [ ]:
def compute_relative_squared_error(actual,pred):
    rse_otu=np.zeros(actual.shape[1],dtype=np.float32)
    actual=actual_array.transpose()
    pred=pred_domain.transpose()
    # for each OTU
    for i, (act_otu,pred_otu) in enumerate(zip(actual,pred)):
        mean_otu = act_otu.mean()
        #display(Markdown("{} (mean(act)): {}</p>".format(i,mean_otu)))
        #display(Markdown("{} (act): {}</p>".format(i,np.array2string(act_otu,precision=8,floatmode='fixed'))))
        #display(Markdown("{} (pred): {}</p>".format(i,np.array2string(pred_otu,precision=8,floatmode='fixed'))))
        div_up=((pred_otu-act_otu)**2).sum()
        div_down=((pred_otu-mean_otu)**2).sum()
        rse_otu[i]=div_up/div_down
    return rse_otu, np.sqrt(rse_otu)
    

In [ ]:
actual_array = transform_to_rel_abundance(data_microbioma_test)
RSE_perOTU, RRSE_perOTU  = compute_relative_squared_error(actual_array,pred_domain)

In [ ]:
RRSE_perOTU.mean()

In [ ]:
RRSE_perOTU.min()

In [ ]:
RRSE_perOTU.max()

#### Save errors per OTU

In [ ]:
def save_errors_per_OTU(RSE,RRSE,otu_names,suffix=''):
    df_RSE = pd.DataFrame(RSE, index=otu_names, columns=['RSE'])
    df_RRSE = pd.DataFrame(RRSE, index=otu_names, columns=['RRSE'])
    df_error = df_RSE.join(df_RRSE)
    df_error.to_csv(suffix+'.tsv', index=True, header=True, sep='\t')
    
    return df_error

In [ ]:
df_error_perOTU = save_errors_per_OTU(RSE_perOTU,RRSE_perOTU,df_microbioma_test.columns,'../Results/errors_perOTU')

In [ ]:
df_error_perOTU

# Novel samples: prediction of novel ecosystems

In [ ]:
metadata = pd.read_csv('../Datasets/NovelEcosystems/metadata_novel_samples_only3envFeatures.csv', sep='\t')
metadata = metadata.set_index('X.SampleID')
domain = metadata[['age',
                   'Temperature',
                   'Precipitation3Days']]
print(domain.shape)


In [ ]:
domain.head()

In [ ]:
domain_novel_samples = domain.to_numpy(dtype=np.float32)

#### Load models

In [ ]:
decoder = tf.keras.models.load_model('../Results/Models/decoder.h5')
encoder_domain = tf.keras.models.load_model('../Results/Models/encoder_domain_model_latent.h5')

#### Compute new predictions

In [ ]:
# Input only domain (i.e. environmental features)
pred_latent_novel_samples = encoder_domain.predict(domain_novel_samples)
pred_domain_novel_samples = decoder.predict(pred_latent_novel_samples)

In [ ]:
df_pred_otu, df_pred_latent = save_predicted_otu_table_and_latent(pred_domain_novel_samples,pred_latent_novel_samples,domain.index,df_microbioma_test.columns,'predFromDomain_novelSamples')